# Waterbomb tessellation demo

In [ ]:
%matplotlib widget
from bmcs_shell.api import WBCell5Param, WBTessellation5PV2, WBNumTessellation, WBNumTessellationInvest, WBTessellationBase, WBNumTessellationBase, WBCell5ParamV2, WBTessellation4P, WBCell4Param
from bmcs_utils.api import Extruder
import numpy as np
import k3d
import sympy as sp

## Analytical tessellation of symmetric waterbomb cell (4P cell)

In [ ]:
def add_circle(plot, path, r):
    n = 100
#     path = np.array([[-4000, 0, -4000], [4000, 0, -4000]])
    first_contour = Extruder.get_circle_points(r = r, n=n)[int(n/2):,:]

    extruder = Extruder(first_contour, path)
    vertices, indices = extruder.get_triangulation_vertices_and_indices(with_ends=False)
    
    # extruder.show_in_k3d_as_surface(with_ends=False)
    
    mesh = k3d.mesh(vertices, indices,
                         color=0xc73737,
                         side='double')
    plot += mesh

In [ ]:
# Cameras for k3d
# [
#  x1,y1,z1, # position of the camera in xyz space
#  x2,y2,z2, # the point where camera is currently looking at
#  x3,y3,z3  # orientation (up direction), this vector cannot be [0,0,0])
# ]
shell_side_view = [9153.397385449189, -7694.631708125074, 1127.588200496582,
                   0.000244140625, -0.00048828125, -2129.4488372802734,
                  -0.20092161158887856, 0.08487577463430307, 0.9759234646614198]
cell_front_view = [0, -2000, 2000,
                   0,     0,   0,
                   0,     0,   1]
shell_front_view = [13500, 0, -1000,
                    0,     0, -2500,
                    0,     0,   1]
shell_top_view = [0, 0, 13500,
                  0, 0, -2500,
                  -1,  0,   0]

In [ ]:
path = np.array([[-4000, 0, -4000], [4000, 0, -4000]])
add_circle(wbt4p.pb.plot_fig, path = path, r=4200)

In [ ]:
# wbt4p.pb.plot_fig.camera = shell_front_view
wbt4p.pb.plot_fig.camera = shell_top_view
# wbt4p.pb.plot_fig.camera = shell_side_view
# wbt4p.pb.plot_fig.camera = cell_front_view

In [ ]:
wbt4p = WBTessellation4P(a = 500, b = 1000, c = 500, gamma = 1.0, n_phi_plus=7, n_x_plus = 5)
wbt4p.interact()
wbt4p.pb.plot_fig.screenshot_scale = 5.5
wbt4p.pb.plot_fig.grid_visible = False
wbt4p.pb.plot_fig.camera = shell_side_view

## Analytical tessellation of 5P cell V2

In [ ]:
# wbttpv2.pb.plot_fig.camera = shell_front_view
wbttpv2.pb.plot_fig.camera = shell_top_view
# wbttpv2.pb.plot_fig.camera = shell_side_view
# wbttpv2.pb.plot_fig.camera = cell_front_view

In [ ]:
path = np.array([[-4000, 0, -4000], [4000, 0, -4000]])
add_circle(wbttpv2.pb.plot_fig, path = path, r=4200)

In [ ]:
# wb_cell=WBCell5ParamV2(a = 500, b = 1000, c = 500, gamma = 1, beta= 1.88)
wb_cell=WBCell5ParamV2(a = 500, b = 1000, c = 500, gamma = 1, beta_1= 0)
wbttpv2 = WBTessellation5PV2(wb_cell=wb_cell, n_x = 9, n_y = 7, sol_num=4)
wbttpv2.interact()
wbttpv2.pb.plot_fig.screenshot_scale = 5.5
wbttpv2.pb.plot_fig.grid_visible = False
wbttpv2.pb.plot_fig.camera = shell_side_view

## General numerical tessellation of any wb cell with 6 facets

In [ ]:
wb_num_t = WBNumTessellation()
wb_num_t.interact()

## Class for investigating 3 cells tessellation manually

In [ ]:
wb_num_t_invest = WBNumTessellationInvest()
wb_num_t_invest.interact()

# Take gif animation from screenshots

In [ ]:
import k3d
import numpy as np
from IPython.display import Image

points_number = 15
spread_range = 30
positions = spread_range * np.random.random_sample((points_number, 3)) - spread_range / 2
colors = np.random.randint(0, 0xFFFFFF, points_number)

plot = k3d.plot(height=240)
points = k3d.points(positions.astype(np.float32), colors.astype(np.uint32), point_size=3.0, shader='mesh')
plot += points

plot

In [ ]:
import ipywidgets

plot.camera_auto_fit = False
plot.grid_auto_fit = False

speeds = -0.5 * np.sign(positions)
out = ipywidgets.Output()

@plot.yield_screenshots
def coroutine():
    global positions
    for i in range(12):
        positions += speeds
        points.positions = positions.astype(np.float32)
        plot.fetch_screenshot()
        screenshot = yield
        with open('screenshot_%03d.png'%i, 'wb') as f:
            f.write(screenshot)
        with out:
            print('screenshot_%03d.png saved.'%i)
    with out:
        print('done.')

coroutine()
print('loop is running in background...')
out